In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29017
2,Huelva,Confirmados PDIA 14 días,1002
3,Huelva,Tasa PDIA 14 días,"195,256932400569"
4,Huelva,Confirmados PDIA 7 días,484
5,Huelva,Total Confirmados,29193
6,Huelva,Curados,25632
7,Huelva,Fallecidos,369


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29017.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7555.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 946 personas en los últimos 7 días 

Un positivo PCR cada 525 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29017.0,484.0,1002.0,513170.0,94.315724,195.256932,101.0
Huelva-Costa,16884.0,234.0,445.0,289548.0,80.815616,153.687817,54.0
Condado-Campiña,9213.0,216.0,480.0,156231.0,138.256812,307.237360,40.0
Huelva (capital),7555.0,152.0,274.0,143837.0,105.675174,190.493406,30.0
Almonte,1596.0,85.0,150.0,24507.0,346.839678,612.070021,17.0
Isla Cristina,2085.0,25.0,40.0,21393.0,116.860655,186.977049,10.0
Cartaya,1288.0,19.0,42.0,20083.0,94.607379,209.132102,6.0
Sierra de Huelva-Andévalo Central,2576.0,32.0,67.0,67391.0,47.484085,99.419804,6.0
Trigueros,273.0,6.0,13.0,7862.0,76.316459,165.352328,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,8.0,1.0,3.0,299.0,334.448161,1003.344482,NaN
Lucena del Puerto,233.0,1.0,27.0,3261.0,30.665440,827.966881,0.0
Almonte,1596.0,85.0,150.0,24507.0,346.839678,612.070021,17.0
Bollullos Par del Condado,861.0,45.0,86.0,14387.0,312.782373,597.761868,5.0
Galaroza,32.0,5.0,7.0,1382.0,361.794501,506.512301,2.0
Rociana del Condado,526.0,11.0,30.0,7939.0,138.556493,377.881345,1.0
Beas,202.0,4.0,15.0,4341.0,92.144667,345.542502,1.0
Condado-Campiña,9213.0,216.0,480.0,156231.0,138.256812,307.237360,40.0
Cumbres de San Bartolomé,11.0,1.0,1.0,376.0,265.957447,265.957447,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,233.0,1.0,27.0,3261.0,30.665440,827.966881,0.0,0.037037
Cortegana,200.0,1.0,6.0,4602.0,21.729683,130.378096,0.0,0.166667
Bonares,257.0,3.0,15.0,6060.0,49.504950,247.524752,0.0,0.200000
San Juan del Puerto,455.0,5.0,22.0,9411.0,53.129317,233.768994,1.0,0.227273
Punta Umbría,718.0,5.0,21.0,15355.0,32.562683,136.763269,0.0,0.238095
Rosal de la Frontera,29.0,1.0,4.0,1697.0,58.927519,235.710077,0.0,0.250000
Beas,202.0,4.0,15.0,4341.0,92.144667,345.542502,1.0,0.266667
Moguer,1118.0,16.0,56.0,21867.0,73.169616,256.093657,2.0,0.285714
Almonaster la Real,66.0,1.0,3.0,1788.0,55.928412,167.785235,0.0,0.333333
